# Statewide Data with Injury Type as Target

## Imports and Setup 

In [1]:
# Initial imports
import pandas as pd
import numpy as np
import psycopg2 
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
# Load in password
from secret import secret, database, username, host

# Establish connection to database
conn = psycopg2.connect(
    host = host,
    database = database,
    user =username,
    password = secret
)

In [3]:
# Load the data
query = 'SELECT * FROM clark_co_traffic'
traffic_df = pd.read_sql_query(query, conn)

/var/folders/f3/5zcd9_1j7d3546ws7g9t5fw00000gp/T/ipykernel_7023/3027837082.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  traffic_df = pd.read_sql_query(query, conn)


In [4]:
# Drop unnecessary columns
traffic_df.drop(columns = ['objectid','crash_severity','accident_rec_num','primary_street', 'secondary_street', 'fatalities', 'injured' ], inplace = True)

In [5]:
# Move PDO values from property_damage_only column info to injury_type column
traffic_df['injury_type'] = np.where(traffic_df['property_damage_only'] == 'PDO', traffic_df['property_damage_only'], traffic_df['injury_type'])

In [6]:
# Drop property damage only column
traffic_df.drop(columns = ['property_damage_only'], inplace = True)

In [7]:
# Split single column with multiple data points into separate columns
traffic_df[['v1_driver_factors_1', 'v1_driver_factors_2','v1_driver_factors_3', 'v1_driver_factors_4']] = traffic_df['v1_driver_factors'].str.split(':', expand=True)

traffic_df[['v1_vehicle_factors_1', 'v1_vehicle_factors_2','v1_vehicle_factors_3', 'v1_vehicle_factors_4', 'v1_vehicle_factors_5']] = traffic_df['v1_vehicle_factors'].str.split(':', expand=True)

traffic_df[['v1_all_events_1', 'v1_all_events_2','v1_all_events_3', 'v1_all_events_4', 'v1_all_events_5']] = traffic_df['v1_all_events'].str.split(':', expand=True)

traffic_df[['v2_driver_factors_1', 'v2_driver_factors_2']] = traffic_df['v2_driver_factors'].str.split(':', expand=True)

traffic_df[['v2_vehicle_factors_1', 'v2_vehicle_factors_2','v2_vehicle_factors_3', 'v2_vehicle_factors_4', 'v2_vehicle_factors_5']] = traffic_df['v2_vehicle_factors'].str.split(':', expand=True)

traffic_df[['v2_all_events_1', 'v2_all_events_2','v2_all_events_3', 'v2_all_events_4', 'v2_all_events_5']] = traffic_df['v1_all_events'].str.split(':', expand=True)

traffic_df[['nonmotorist_factors_1', 'nonmotorist_factors_2','nonmotorist_factors_3', 'nonmotorist_factors_4', 'nonmotorist_factors_5']] = traffic_df['nonmotorist_factors'].str.split(':', expand=True)

traffic_df[['factors_roadway_1', 'factors_roadway_2','factors_roadways_3']] = traffic_df['factors_roadway'].str.split(':', expand=True)

traffic_df[['hwy_factors_1', 'hwy_factors_2','hwy_factors_3', 'hwy_factors_4']] = traffic_df['hwy_factors'].str.split(':', expand=True)

traffic_df.drop(columns = ['v1_driver_factors','v1_vehicle_factors','v1_all_events','v2_driver_factors','v2_vehicle_factors', 'v2_all_events', 'nonmotorist_factors', 'factors_roadway', 'hwy_factors',], inplace = True)

In [8]:
traffic_df['injury_type'].value_counts()

PDO    128261
C       69561
B       22594
A        3836
K        1175
U         241
Name: injury_type, dtype: int64

In [9]:
# Change target values to numeric
# Values are based on Nevada Traffic Records Coordinating Committee Data Dictionary,
# https://zerofatalitiesnv.com/app/uploads/2021/04/2021-01-NV-TRCC-Data-Dictionary.pdf
injury_num = {
    'K': 5,
    'A': 4,
    'B': 3,
    'C': 2,
    'PDO': 1,
    'U': 0,    
}

traffic_df['injury_type'] = traffic_df['injury_type'].apply(lambda x: injury_num[x])

traffic_df['injury_type'].value_counts()

1    128261
2     69561
3     22594
4      3836
5      1175
0       241
Name: injury_type, dtype: int64

In [10]:
# Extract datetime values into separate columns
traffic_df['crash_month'] = pd.DatetimeIndex(traffic_df['crash_date']).month
traffic_df['crash_day'] = pd.DatetimeIndex(traffic_df['crash_date']).day
traffic_df['crash_day_of_week'] = pd.DatetimeIndex(traffic_df['crash_date']).dayofweek
traffic_df['crash_hour'] = pd.DatetimeIndex(traffic_df['crash_time']).hour


In [11]:
# Drop original datetime columns
traffic_df.drop(columns = ['crash_date', 'crash_time'], inplace = True)

In [12]:
traffic_df.dtypes

x                    float64
y                    float64
county                object
crash_year             int64
weather               object
                      ...   
hwy_factors_4         object
crash_month            int64
crash_day              int64
crash_day_of_week      int64
crash_hour             int64
Length: 65, dtype: object

In [13]:
# Get names of all columns with dtypes of "object"
sel_cols = list(traffic_df.select_dtypes(include='object'))


In [14]:
# Preprocessing to convert all strings to numeric values
traffic_df_encoded = pd.get_dummies(traffic_df, columns = sel_cols)
traffic_df_encoded.head()

,x,y,crash_year,injury_type,total_vehicles,v1_driver_age,v2_driver_age,crash_month,crash_day,crash_day_of_week,...,hwy_factors_3_ OTHER HIGHWAY,hwy_factors_3_ ROAD OBSTRUCTION,"hwy_factors_3_ RUTS, HOLES, BUMPS",hwy_factors_3_ SHOULDERS,hwy_factors_3_ VISUAL OBSTRUCTION(S),hwy_factors_3_ WEATHER,"hwy_factors_3_ WET, ICY, SNOW, SLUSH",hwy_factors_3_ WORK ZONE (CONST. MAINT. UTILITY),hwy_factors_3_ WORN TRAFFIC SURFACE,"hwy_factors_4_ WET, ICY, SNOW, SLUSH"
0,-115.106709,36.236043,2016,1,2,35,41,1,4,0,...,0,0,0,0,0,0,0,0,0,0
1,-119.673794,39.626433,2016,1,1,39,41,1,7,3,...,0,0,0,0,0,0,0,0,0,0
2,-115.101063,36.240200,2016,1,2,31,53,1,4,0,...,0,0,0,0,0,0,0,0,0,0
3,-119.633092,39.516952,2016,1,1,18,41,1,6,2,...,0,0,0,0,0,0,0,0,0,0
4,-115.140570,36.192680,2016,1,1,56,41,1,3,6,...,0,0,0,0,0,0,0,0,0,0


## Model with Injury Type Crash Severity as Target and All Other Columns as Features

In [15]:
# Separate data into features and target
y = traffic_df_encoded['injury_type'].values
X = traffic_df_encoded.drop(columns = 'injury_type')

In [16]:
# Split dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y)
X_train.shape

(169251, 1345)

In [17]:
# Scale the model
scaler = StandardScaler() 

# Fit the scaler with the training data
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [18]:
# Create and fit the Random Forest model
rf_model = RandomForestClassifier(n_estimators = 128)
rf_model = rf_model.fit(X_train_scaled, y_train)

In [19]:
# Create predictions
predictions = rf_model.predict(X_test_scaled)
results = pd.DataFrame({
    "Prediction": predictions,
    "Actual": y_test
}).reset_index(drop = True)
results.head()

,Prediction,Actual
0,1,1
1,1,3
2,2,3
3,1,1
4,1,2


In [20]:
# Assess accuracy score
accuracy = accuracy_score(y_test, predictions)
accuracy

0.656681496711984

In [21]:
# Generate confusion matrix and dataframe
cm = confusion_matrix(y_test, predictions)

index_values = ['Actual: Unknown', 
                'Actual: Property Damage Only', 
                'Actual: Possible Injury', 
                'Actual: Suspected Minor Injury', 
                'Actual: Suspected Serious Injury', 
                'Actual: Fatal Injury']

column_values = ['Predicted: Unknown', 
                'Predicted: Property Damage Only', 
                'Predicted: Possible Injury', 
                'Predicted: Suspected Minor Injury', 
                'Predicted: Suspected Serious Injury', 
                'Predicted: Fatal Injury']

cm_df = pd.DataFrame(cm, index = index_values, columns = column_values)
cm_df
                  

,Predicted: Unknown,Predicted: Property Damage Only,Predicted: Possible Injury,Predicted: Suspected Minor Injury,Predicted: Suspected Serious Injury,Predicted: Fatal Injury
Actual: Unknown,0,33,13,14,0,0
Actual: Property Damage Only,0,27241,4690,130,4,0
Actual: Possible Injury,0,7787,9122,466,15,0
Actual: Suspected Minor Injury,0,2238,2760,618,32,1
Actual: Suspected Serious Injury,0,327,367,229,28,8
Actual: Fatal Injury,0,111,68,64,12,39


## Results with Injury Type as Target and All Other Columns as Features

In [22]:
# Display results
print('Statewide Data with Injury Type as Target, All Other Columns as Features')
print('Confusion Matrix')
display(cm_df)
print(f'\nAccuracy Score: {accuracy}\n')
print('Classification Report')
print(classification_report(y_test, predictions))

Statewide Data with Injury Type as Target, All Other Columns as Features
Confusion Matrix


,Predicted: Unknown,Predicted: Property Damage Only,Predicted: Possible Injury,Predicted: Suspected Minor Injury,Predicted: Suspected Serious Injury,Predicted: Fatal Injury
Actual: Unknown,0,33,13,14,0,0
Actual: Property Damage Only,0,27241,4690,130,4,0
Actual: Possible Injury,0,7787,9122,466,15,0
Actual: Suspected Minor Injury,0,2238,2760,618,32,1
Actual: Suspected Serious Injury,0,327,367,229,28,8
Actual: Fatal Injury,0,111,68,64,12,39



Accuracy Score: 0.656681496711984

Classification Report
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        60
           1       0.72      0.85      0.78     32065
           2       0.54      0.52      0.53     17390
           3       0.41      0.11      0.17      5649
           4       0.31      0.03      0.05       959
           5       0.81      0.13      0.23       294

    accuracy                           0.66     56417
   macro avg       0.46      0.27      0.29     56417
weighted avg       0.63      0.66      0.63     56417



/opt/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Importances with Injury Type as Target and All Other Columns as Features

In [23]:
# Calculate importance of features
importances = rf_model.feature_importances_
important = sorted(zip(importances, X.columns), reverse = True)
important

[(0.05356613573406276, 'y'),
 (0.05310038479234736, 'x'),
 (0.05225500779004965, 'v1_driver_age'),
 (0.044745962758455736, 'crash_day'),
 (0.044365084666128, 'v2_driver_age'),
 (0.04187348521697159, 'crash_hour'),
 (0.03855902652295371, 'crash_month'),
 (0.03416299673833816, 'crash_day_of_week'),
 (0.022529982005295608, 'crash_year'),
 (0.019371275169995626, 'factors_roadway_1_UNKNOWN'),
 (0.016935661551530354, 'lighting_UNKNOWN'),
 (0.016070220290711607, 'factors_roadway_1_DRY'),
 (0.012506647867905094, 'hwy_factors_1_NONE'),
 (0.012373593307631846, 'lighting_DAYLIGHT'),
 (0.012261277510306963, 'hwy_factors_1_UNKNOWN'),
 (0.012153271193954804, 'total_vehicles'),
 (0.011253591143717961, 'v1_type_SEDAN, 4 DOOR'),
 (0.008826600097254837, 'v1_type_CARRY-ALL'),
 (0.0087819736882828, 'v2_type_SEDAN, 4 DOOR'),
 (0.00844476731630732, 'v1_action_GOING STRAIGHT'),
 (0.008099594984237036, 'v1_driver_factors_1_APPARENTLY NORMAL'),
 (0.00789721788453277, 'v1_type_PICKUP'),
 (0.007672067614482831, 

## New Version of Model Based on Top Importances

### Importances to exclude

In [24]:
low_priority = important[772:]


In [25]:
imp_only_list = []
listy_list = []

# iterate using index with enumerate function
for index, tuple in enumerate(low_priority):
       
    # access through index by appending to list
    imp_only_list.append(low_priority[index])
     
# iterate through the list
for x in imp_only_list:
    for y in x:
         listy_list.append(y)

final_list = listy_list[1::2]



### Statewide Dataframe without Low-Value Importances

In [26]:
important_df = traffic_df_encoded.drop(columns = final_list)
important_df.head()

,x,y,crash_year,injury_type,total_vehicles,v1_driver_age,v2_driver_age,crash_month,crash_day,crash_day_of_week,...,hwy_factors_2_ BACKUP DUE TO PRIOR NON-RECURRING INCIDENT,hwy_factors_2_ BACKUP DUE TO REGULAR CONGESTION,hwy_factors_2_ DEBRIS,hwy_factors_2_ OTHER HIGHWAY,"hwy_factors_2_ WET, ICY, SNOW, SLUSH",hwy_factors_2_ WORK ZONE (CONST. MAINT. UTILITY),hwy_factors_3_ BACKUP DUE TO PRIOR CRASH,hwy_factors_3_ BACKUP DUE TO REGULAR CONGESTION,hwy_factors_3_ OTHER HIGHWAY,hwy_factors_3_ WORK ZONE (CONST. MAINT. UTILITY)
0,-115.106709,36.236043,2016,1,2,35,41,1,4,0,...,0,0,0,0,0,0,0,0,0,0
1,-119.673794,39.626433,2016,1,1,39,41,1,7,3,...,0,0,0,0,0,0,0,0,0,0
2,-115.101063,36.240200,2016,1,2,31,53,1,4,0,...,0,0,0,0,0,0,0,0,0,0
3,-119.633092,39.516952,2016,1,1,18,41,1,6,2,...,0,0,0,0,0,0,0,0,0,0
4,-115.140570,36.192680,2016,1,1,56,41,1,3,6,...,0,0,0,0,0,0,0,0,0,0


## Model with Injury Type as Target and Only High-Value Importances as Features

In [27]:
# Separate data into features and target
y_imp = important_df['injury_type'].values
X_imp = important_df.drop(columns = 'injury_type')

In [28]:
X_train_imp, X_test_imp, y_train_imp, y_test_imp = train_test_split(X_imp, y_imp)
X_train_imp.shape

(169251, 772)

In [29]:
# Scale the model
scaler_imp = StandardScaler() 

# Fit the scaler with the training data
X_scaler_imp = scaler_imp.fit(X_train_imp)
X_train_scaled_imp = X_scaler_imp.transform(X_train_imp)
X_test_scaled_imp = X_scaler_imp.transform(X_test_imp)

In [30]:
# Create and fit the Random Forest model
rf_model_imp = RandomForestClassifier(n_estimators = 128)
rf_model_imp = rf_model.fit(X_train_scaled_imp, y_train_imp)

In [31]:
# Create predictions
predictions_imp = rf_model_imp.predict(X_test_scaled_imp)
results_imp = pd.DataFrame({
    "Prediction": predictions_imp,
    "Actual": y_test_imp
}).reset_index(drop = True)
results_imp.head()

,Prediction,Actual
0,1,1
1,1,1
2,1,5
3,1,1
4,1,1


In [32]:
# Assess accuracy score
accuracy_imp = accuracy_score(y_test_imp, predictions_imp)
accuracy_imp

0.6553875604870872

In [33]:
# Generate confusion matrix and dataframe
cm_imp = confusion_matrix(y_test_imp, predictions_imp)

index_values_imp = ['Actual: Unknown', 
                'Actual: Property Damage Only', 
                'Actual: Possible Injury', 
                'Actual: Suspected Minor Injury', 
                'Actual: Suspected Serious Injury', 
                'Actual: Fatal Injury']

column_values_imp = ['Predicted: Unknown', 
                'Predicted: Property Damage Only', 
                'Predicted: Possible Injury', 
                'Predicted: Suspected Minor Injury', 
                'Predicted: Suspected Serious Injury', 
                'Predicted: Fatal Injury']

cm_df_imp = pd.DataFrame(cm_imp, index = index_values_imp, columns = column_values_imp)
cm_df_imp

,Predicted: Unknown,Predicted: Property Damage Only,Predicted: Possible Injury,Predicted: Suspected Minor Injury,Predicted: Suspected Serious Injury,Predicted: Fatal Injury
Actual: Unknown,0,31,13,14,0,2
Actual: Property Damage Only,0,27155,4554,127,5,1
Actual: Possible Injury,0,7975,9066,496,18,1
Actual: Suspected Minor Injury,0,2221,2733,696,21,0
Actual: Suspected Serious Injury,0,356,357,222,27,8
Actual: Fatal Injury,0,143,61,68,15,31


## Statewide Results with Injury Type as Target and High-Priority Columns as Features

In [34]:
# Display results
print('Statewide Data with Injury Type as Target, High-Priority Columns as Features')
print('Confusion Matrix')
display(cm_df_imp)
print(f'\nAccuracy Score: {accuracy_imp}\n')
print('Classification Report')
print(classification_report(y_test_imp, predictions_imp))

Statewide Data with Injury Type as Target, High-Priority Columns as Features
Confusion Matrix


,Predicted: Unknown,Predicted: Property Damage Only,Predicted: Possible Injury,Predicted: Suspected Minor Injury,Predicted: Suspected Serious Injury,Predicted: Fatal Injury
Actual: Unknown,0,31,13,14,0,2
Actual: Property Damage Only,0,27155,4554,127,5,1
Actual: Possible Injury,0,7975,9066,496,18,1
Actual: Suspected Minor Injury,0,2221,2733,696,21,0
Actual: Suspected Serious Injury,0,356,357,222,27,8
Actual: Fatal Injury,0,143,61,68,15,31



Accuracy Score: 0.6553875604870872

Classification Report
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        60
           1       0.72      0.85      0.78     31842
           2       0.54      0.52      0.53     17556
           3       0.43      0.12      0.19      5671
           4       0.31      0.03      0.05       970
           5       0.72      0.10      0.17       318

    accuracy                           0.66     56417
   macro avg       0.45      0.27      0.29     56417
weighted avg       0.63      0.66      0.62     56417



/opt/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Calculate importance of features

In [35]:

importances_high_priority = rf_model_imp.feature_importances_
sorted(zip(importances_high_priority, X_imp.columns), reverse = True)

[(0.0546261394421261, 'y'),
 (0.054258952890922446, 'x'),
 (0.052082924206990996, 'v1_driver_age'),
 (0.04552286649154422, 'crash_day'),
 (0.045158221183857136, 'v2_driver_age'),
 (0.042438512641388995, 'crash_hour'),
 (0.03861532504171736, 'crash_month'),
 (0.03414168014918644, 'crash_day_of_week'),
 (0.022141856654509372, 'crash_year'),
 (0.018936924096718946, 'factors_roadway_1_UNKNOWN'),
 (0.017566130789234136, 'lighting_UNKNOWN'),
 (0.014074395135518163, 'hwy_factors_1_UNKNOWN'),
 (0.013644395920909162, 'factors_roadway_1_DRY'),
 (0.012917918317766274, 'hwy_factors_1_NONE'),
 (0.012698803801902777, 'lighting_DAYLIGHT'),
 (0.012185201695798774, 'total_vehicles'),
 (0.011546324190486084, 'v1_type_SEDAN, 4 DOOR'),
 (0.008976360218298418, 'v1_type_CARRY-ALL'),
 (0.008274188540074402, 'v1_driver_factors_1_APPARENTLY NORMAL'),
 (0.008252249661665878, 'v2_type_SEDAN, 4 DOOR'),
 (0.008145045734323229, 'v1_action_GOING STRAIGHT'),
 (0.008022377417598972, 'v1_vehicle_factors_1_FAILED TO YIE